In [1]:
!pip install bayesian-optimization

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization

2024-02-14 14:17:40.323088: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-14 14:17:40.607395: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-14 14:17:40.607446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-14 14:17:40.653363: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-14 14:17:40.746598: I tensorflow/core/platform/cpu_feature_guar

In [3]:

# Load and preprocess the data
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
# Define the neural network architecture
def create_model(learning_rate, num_hidden_layers, num_neurons):
    model = tf.keras.Sequential()
    for i in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons, activation='relu'))
    model.add(tf.keras.layers.Dense(2, activation='softmax'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [5]:
# Define the objective function to optimize
def objective(learning_rate, num_hidden_layers, num_neurons):
    model = create_model(learning_rate, int(num_hidden_layers), int(num_neurons))
    model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=0)
    val_loss, val_acc = model.evaluate(X_test, y_test, verbose=0)
    return val_acc

In [6]:
# Define the search space for hyperparameters
pbounds = {'learning_rate': (0.0001, 0.1),
           'num_hidden_layers': (1, 5),
           'num_neurons': (5, 50)}

In [7]:
# Perform Bayesian optimization
optimizer = BayesianOptimization(f=objective, pbounds=pbounds, verbose=2)
optimizer.maximize(init_points=5, n_iter=20)

|   iter    |  target   | learni... | num_hi... | num_ne... |
-------------------------------------------------------------
| 1         | 0.9825    | 0.09941   | 2.999     | 18.17     |
| 2         | 0.9649    | 0.004418  | 4.421     | 45.5      |
| 3         | 0.9649    | 0.04627   | 3.873     | 35.88     |
| 4         | 0.9825    | 0.09455   | 2.837     | 33.53     |
| 5         | 0.9649    | 0.02331   | 4.438     | 12.48     |
| 6         | 0.9825    | 0.0811    | 3.991     | 21.06     |
| 7         | 0.9825    | 0.02152   | 1.646     | 10.72     |
| 8         | 0.9649    | 0.07627   | 3.959     | 21.07     |
| 9         | 0.9825    | 0.01397   | 3.071     | 35.43     |
| 10        | 0.9649    | 0.01386   | 2.39      | 28.73     |
| 11        | 0.9737    | 0.04459   | 3.533     | 15.98     |
| 12        | 0.9649    | 0.0919    | 4.574     | 30.29     |
| 13        | 0.9737    | 0.05057   | 2.749     | 47.06     |
| 14        | 0.9649    | 0.01725   | 2.286     | 26.61     |
| 15    

In [8]:
# Print the optimized hyperparameters and validation accuracy
print('Optimized hyperparameters:')
print(optimizer.max['params'])
print('Validation accuracy: {:.2f}%'.format(optimizer.max['target'] * 100))

Optimized hyperparameters:
{'learning_rate': 0.09940764465149246, 'num_hidden_layers': 2.9994850555888997, 'num_neurons': 18.172399993787913}
Validation accuracy: 98.25%
